# Redes Neuronales Recurrentes
## Paso 1: Importacion

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%matplotlib inline

In [2]:
from tensorflow.keras.datasets import imdb
tf.__version__

'2.2.0'

## Paso 2: Preprocesado de datos
Para estos tipos de casos, con lo que estamos trabajando es palabras y no pixeles como vimos en casos anterior. Por lo que el pre-procesado de datos va a ser diferente.
### Configuracion parametros del dataset
Como quiero que todas las valoraciones tengan la misma longitud voy a necesitar un nuevo metodo de trabajo.

In [0]:
number_of_words = 20000
max_len = 100
## todas van a tener la misma longitud

### Carga de dataset IMDB

In [4]:
(x_train , y_train), (x_test , y_test) = imdb.load_data(num_words=number_of_words)
# busco las 20.000 palabras mas frecuentes

17465344/17464789 [==============================] - 1s 0us/step


### Cortar secuencia de textos de la misma longitud

In [5]:
x_train.shape , y_train.shape

((25000,), (25000,))

In [6]:
len(x_train[0])

218

In [0]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

In [8]:
len(x_train[0])

100

### Configurar parametros de la capa de Embedding

In [9]:
vocabulary_size = number_of_words
vocabulary_size

20000

In [0]:
embed_size = 128
#128 pone el profe por defecto
#300 dio un acurrancy de 0.85
#60 dio un acurrancy de 0.84

## Paso 3: Contruccion del Modelo

### Definir Modelo

In [0]:
model = tf.keras.Sequential()

### Añadir capa de embedding

En aprendizaje automático, embedding (incrustrar) es un término especial que simplemente significa proyectar una entrada en otro espacio de representación más conveniente.

https://www.quora.com/What-does-the-word-embedding-mean-in-the-context-of-Machine-Learning

Me va a permitir conservar los valores de entrada y comprender la relaciones que hay entre las palabras.

output_dim = va a determinar el tamanio de los embendings
* Al aumentar su tamanio se va a necesitar hacer mucha mas cantidad de calculos.

In [0]:
model.add(tf.keras.layers.Embedding(input_dim=vocabulary_size , output_dim=embed_size , input_shape=(x_train.shape[1],)))

### Añadir capa LSTM
* Unidades: 128
  * Es la salida del embendding
* funcion de activacion tanh
  * Esto es devido al funcionamiento natural de este tipo de neuronas

In [0]:
model.add(tf.keras.layers.LSTM(units=128 , activation='tanh'))

### Añadir la capa totalmente conectada de salida
* units : 1
* func_act : sigmoid
  * Probabilidad entre 0 y 1

In [0]:
#model.add(tf.keras.layers.Dense(units=16, activation='relu')) con esto y dos epocs --> 0.85
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
#model.add(tf.keras.layers.Dropout(0.2)) para nada recomendable --> 0.82 de exito genero

## Compilar Modelo

rmsprop : Proporcion del error cuadrado medio. Es la mas recomendada para redes neuronales recurrentes.

https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a 

In [0]:
model.compile(optimizer='rmsprop' , loss='binary_crossentropy' , metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Entrenar modelo

In [17]:
model.fit(x_train,y_train, epochs=2 , batch_size=128) 
#0.82 de exactitud con un bach de 128
#0.85 de exactitud con un bach de 300
#Con un ephoc de 2 y batch de 128, ca un acurrancy de 0.85

Epoch 1/2
196/196 [==============================] - 5s 27ms/step - loss: 0.4536 - accuracy: 0.7935
Epoch 2/2
196/196 [==============================] - 5s 26ms/step - loss: 0.2867 - accuracy: 0.8866


In [18]:
test_loss , test_acurracy = model.evaluate(x_test, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3583 - accuracy: 0.8448


In [19]:
print('Acurracy: ',test_acurracy)

Acurracy:  0.8448399901390076
